In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
'''
ToTensor() 将图像转换为张量
Normalize()将图像标准化（-1，1） output[channel] = (input[channel] - mean[channel]) / std[channel]
'''
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(std=(0.5, 0.5, 0.5), mean=(0.5, 0.5, 0.5))
])

In [3]:
trainset = torchvision.datasets.CIFAR10(
    root='../../resources/data/', 
    train=True, 
    transform=transform, 
    download=True
)

Files already downloaded and verified


In [4]:
trainloader = torch.utils.data.DataLoader(
    trainset, 
    batch_size=4, 
    shuffle=True, 
    num_workers=4
)

In [5]:
testset = torchvision.datasets.CIFAR10(
    root='../../resources/data/',
    train=False,
    transform=transform,
    download=True
)

Files already downloaded and verified


In [6]:
testlodaer = torch.utils.data.DataLoader(
    testset,
    batch_size=4,
    shuffle=False,
    num_workers=4
)

In [7]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [8]:
# 定义神经网络
import torch.nn as nn
import torch.nn.functional as F

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cup')

In [12]:
net = Net()
net = net.to(device)

In [13]:
# 定义损失函数和优化器
import torch.optim as optim

In [14]:
criterion = nn.CrossEntropyLoss() # 使用交叉熵作为损失函数

In [16]:
criterion = criterion.to(device)

In [17]:
# 使用带动量的随机梯度下降做优化器
optimizer = optim.SGD(params=net.parameters(), lr=0.001, momentum=0.9) 

In [18]:
# 训练网络
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()  # 梯度置零
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # 打印训练状态
        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d %5d] loss: %.3f' % (epoch+1, i+1, running_loss/2000))
            running_loss = 0.0
print('-------------FINISHED TRAINING---------------')

[1  2000] loss: 2.200
[1  4000] loss: 1.895
[1  6000] loss: 1.722
[1  8000] loss: 1.603
[1 10000] loss: 1.546
[1 12000] loss: 1.494
[2  2000] loss: 1.433
[2  4000] loss: 1.419
[2  6000] loss: 1.388
[2  8000] loss: 1.373
[2 10000] loss: 1.347
[2 12000] loss: 1.307
-------------FINISHED TRAINING---------------
